# BERT EVALUATION

### IMPORTS, VARIABLES

In [1]:
import conllu
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import time

pd.set_option('display.max_rows', None)

In [2]:
xpos_predictions = './bert/polXPOS-model/test_predictions.txt'
upos_predictions = './bert/polUPOS-model/test_predictions.txt'
file_3k = '../data/memoirs_3k_corrected.conllu'
file_10k = '../data/memoirs_10k_corrected.conllu'

labels_xpos = './bert/data_XPOS/labels.txt'
labels_upos = './bert/data_UPOS/labels.txt'

In [3]:
from functions import extract_conllu_data, get_measures

### FUNCTIONS

In [4]:
def get_labels(filename: str):
    with open(filename) as f:
        labels = f.readlines()[1:]
        labels = [x.strip() for x in labels]
        
    return labels

### EXECUTION

In [5]:
labels_upos_list = get_labels(labels_upos)
labels_xpos_list = get_labels(labels_xpos)

In [6]:
upos_standard_list = extract_conllu_data(file_10k,'upos', sentences=False)
xpos_standard_list = extract_conllu_data(file_3k, 'xpos', sentences=False)

ParseException: Failed parsing field 'id': '0-1' is not a valid ID.